In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/movie-recommendation/movies.csv
/kaggle/input/movie-recommendation/ratings.csv
/kaggle/input/movie-recommendation/README.txt
/kaggle/input/movie-recommendation/tags.csv
/kaggle/input/movie-recommendation/links.csv


In [1]:
!pip uninstall -y numpy
!pip install "numpy<2"


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 82.4 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.47.0 which is incompatible.
google-colab 1.0.0 requires jupyter-server==2.14.0, but you have jupyter-server 2.12.5 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you

In [3]:
!pip install --force-reinstall pandas scipy scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 87.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 60.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 82.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 13.6 MB/s eta 0:00:00
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2554977 sha256=e101ce6a8b6fd2595123dee1bbe206d508a6109d10cf46960276c62d527e49c9
  Stored in

In [1]:
"""
ReelSense: Explainable Movie Recommender System
Kaggle Training Pipeline
"""

# ============================================================================
# PART 1: IMPORTS AND SETUP
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

# Surprise library for Matrix Factorization
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

# For saving models
import pickle
import json

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All libraries imported successfully!")

# ============================================================================
# PART 2: DATA LOADING
# ============================================================================

# Load datasets
ratings_df = pd.read_csv('/kaggle/input/movie-recommendation/ratings.csv')
movies_df = pd.read_csv('/kaggle/input/movie-recommendation/movies.csv')
tags_df = pd.read_csv('/kaggle/input/movie-recommendation/tags.csv')
links_df = pd.read_csv('/kaggle/input/movie-recommendation/links.csv')

print("📊 Dataset Statistics:")
print(f"Ratings: {len(ratings_df):,} rows")
print(f"Movies: {len(movies_df):,} rows")
print(f"Tags: {len(tags_df):,} rows")
print(f"Links: {len(links_df):,} rows")
print(f"\nUsers: {ratings_df['userId'].nunique():,}")
print(f"Movies rated: {ratings_df['movieId'].nunique():,}")

# Display samples
print("\n📋 Sample Data:")
print("\nRatings:")
print(ratings_df.head())
print("\nMovies:")
print(movies_df.head())
print("\nTags:")
print(tags_df.head())

# ============================================================================
# PART 3: DATA PREPROCESSING
# ============================================================================

print("\n🧹 PREPROCESSING DATA...")

# Convert timestamp to datetime
ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'], unit='s')
if 'timestamp' in tags_df.columns:
    tags_df['timestamp'] = pd.to_datetime(tags_df['timestamp'], unit='s')

# Sort by timestamp
ratings_df = ratings_df.sort_values('timestamp')

# Clean genres
movies_df['genres_list'] = movies_df['genres'].str.split('|')
movies_df['genres_clean'] = movies_df['genres'].str.replace('|', ' ')

# Clean tags
if len(tags_df) > 0:
    tags_df['tag'] = tags_df['tag'].str.lower().str.strip()
    
    # Aggregate tags per movie
    movie_tags = tags_df.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()
    movie_tags.columns = ['movieId', 'tags_combined']
    
    # Merge with movies
    movies_df = movies_df.merge(movie_tags, on='movieId', how='left')
    movies_df['tags_combined'] = movies_df['tags_combined'].fillna('')
else:
    movies_df['tags_combined'] = ''

# Create content features (genres + tags)
movies_df['content_features'] = movies_df['genres_clean'] + ' ' + movies_df['tags_combined']

# Extract year from title
movies_df['year'] = movies_df['title'].str.extract(r'\((\d{4})\)').astype(float)

print("✅ Preprocessing complete!")

# ============================================================================
# PART 4: EXPLORATORY DATA ANALYSIS
# ============================================================================

print("\n📊 PERFORMING EDA...")

# Create EDA directory
import os
os.makedirs('eda_plots', exist_ok=True)

# 1. Rating Distribution
plt.figure(figsize=(10, 6))
ratings_df['rating'].value_counts().sort_index().plot(kind='bar', color='steelblue')
plt.title('Distribution of Ratings', fontsize=16, fontweight='bold')
plt.xlabel('Rating', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('eda_plots/rating_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

# 2. User Activity Distribution
plt.figure(figsize=(12, 6))
user_activity = ratings_df['userId'].value_counts()
plt.hist(user_activity, bins=50, color='coral', edgecolor='black')
plt.title('User Activity Distribution', fontsize=16, fontweight='bold')
plt.xlabel('Number of Ratings per User', fontsize=12)
plt.ylabel('Number of Users', fontsize=12)
plt.axvline(user_activity.mean(), color='red', linestyle='--', label=f'Mean: {user_activity.mean():.1f}')
plt.legend()
plt.tight_layout()
plt.savefig('eda_plots/user_activity.png', dpi=300, bbox_inches='tight')
plt.close()

# 3. Movie Popularity (Long-tail)
plt.figure(figsize=(12, 6))
movie_counts = ratings_df['movieId'].value_counts()
plt.plot(range(len(movie_counts)), sorted(movie_counts.values, reverse=True), color='darkgreen')
plt.title('Long-tail Distribution of Movie Popularity', fontsize=16, fontweight='bold')
plt.xlabel('Movie Rank', fontsize=12)
plt.ylabel('Number of Ratings', fontsize=12)
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('eda_plots/movie_longtail.png', dpi=300, bbox_inches='tight')
plt.close()

# 4. Genre Popularity
all_genres = []
for genres_list in movies_df['genres_list']:
    if isinstance(genres_list, list):
        all_genres.extend(genres_list)

genre_counts = pd.Series(all_genres).value_counts().head(15)

plt.figure(figsize=(12, 6))
genre_counts.plot(kind='barh', color='purple')
plt.title('Top 15 Most Popular Genres', fontsize=16, fontweight='bold')
plt.xlabel('Count', fontsize=12)
plt.ylabel('Genre', fontsize=12)
plt.tight_layout()
plt.savefig('eda_plots/genre_popularity.png', dpi=300, bbox_inches='tight')
plt.close()

# 5. Average Rating by Genre
genre_ratings = {}
for idx, row in movies_df.iterrows():
    if isinstance(row['genres_list'], list):
        movie_ratings = ratings_df[ratings_df['movieId'] == row['movieId']]['rating']
        if len(movie_ratings) > 0:
            avg_rating = movie_ratings.mean()
            for genre in row['genres_list']:
                if genre not in genre_ratings:
                    genre_ratings[genre] = []
                genre_ratings[genre].append(avg_rating)

genre_avg = {k: np.mean(v) for k, v in genre_ratings.items() if len(v) > 10}
genre_avg_series = pd.Series(genre_avg).sort_values(ascending=False)

plt.figure(figsize=(12, 6))
genre_avg_series.plot(kind='barh', color='teal')
plt.title('Average Rating by Genre', fontsize=16, fontweight='bold')
plt.xlabel('Average Rating', fontsize=12)
plt.ylabel('Genre', fontsize=12)
plt.tight_layout()
plt.savefig('eda_plots/genre_avg_rating.png', dpi=300, bbox_inches='tight')
plt.close()

# 6. Rating Trends Over Time
ratings_df['year_rated'] = ratings_df['timestamp'].dt.year
yearly_stats = ratings_df.groupby('year_rated').agg({
    'rating': ['mean', 'count']
}).reset_index()
yearly_stats.columns = ['year', 'avg_rating', 'count']

fig, ax1 = plt.subplots(figsize=(12, 6))
ax2 = ax1.twinx()

ax1.plot(yearly_stats['year'], yearly_stats['avg_rating'], 'b-o', label='Avg Rating')
ax2.bar(yearly_stats['year'], yearly_stats['count'], alpha=0.3, color='orange', label='Count')

ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Average Rating', fontsize=12, color='b')
ax2.set_ylabel('Number of Ratings', fontsize=12, color='orange')
ax1.set_title('Rating Trends Over Time', fontsize=16, fontweight='bold')
ax1.tick_params(axis='y', labelcolor='b')
ax2.tick_params(axis='y', labelcolor='orange')
plt.tight_layout()
plt.savefig('eda_plots/rating_trends.png', dpi=300, bbox_inches='tight')
plt.close()

print("✅ EDA plots saved to 'eda_plots/' directory")

# ============================================================================
# PART 5: TRAIN-TEST SPLIT (LEAVE-LAST-N)
# ============================================================================

print("\n🔀 CREATING TRAIN-TEST SPLIT...")

# Leave-last-1 per user
def leave_last_n_split(df, n=1):
    """Split data leaving last N ratings per user for testing"""
    train_list = []
    test_list = []
    
    for user_id in df['userId'].unique():
        user_data = df[df['userId'] == user_id].sort_values('timestamp')
        
        if len(user_data) > n:
            train_list.append(user_data.iloc[:-n])
            test_list.append(user_data.iloc[-n:])
        else:
            train_list.append(user_data)
    
    train_df = pd.concat(train_list, ignore_index=True)
    test_df = pd.concat(test_list, ignore_index=True)
    
    return train_df, test_df

train_ratings, test_ratings = leave_last_n_split(ratings_df, n=1)

print(f"Train set: {len(train_ratings):,} ratings")
print(f"Test set: {len(test_ratings):,} ratings")
print(f"Train users: {train_ratings['userId'].nunique()}")
print(f"Test users: {test_ratings['userId'].nunique()}")

# ============================================================================
# PART 6: MODEL 1 - POPULARITY-BASED RECOMMENDER
# ============================================================================

print("\n🌟 BUILDING POPULARITY-BASED MODEL...")

class PopularityRecommender:
    def __init__(self):
        self.popular_movies = None
        self.movie_data = None
    
    def fit(self, ratings_df, movies_df):
        """Train popularity model"""
        movie_stats = ratings_df.groupby('movieId').agg({
            'rating': ['count', 'mean']
        }).reset_index()
        movie_stats.columns = ['movieId', 'rating_count', 'rating_mean']
        
        # Filter movies with at least 50 ratings
        movie_stats = movie_stats[movie_stats['rating_count'] >= 50]
        
        # Calculate popularity score (weighted rating)
        C = movie_stats['rating_mean'].mean()
        m = movie_stats['rating_count'].quantile(0.75)
        
        movie_stats['popularity_score'] = (
            (movie_stats['rating_count'] / (movie_stats['rating_count'] + m)) * movie_stats['rating_mean'] +
            (m / (movie_stats['rating_count'] + m)) * C
        )
        
        # Merge with movie data
        self.popular_movies = movie_stats.merge(movies_df, on='movieId')
        self.popular_movies = self.popular_movies.sort_values('popularity_score', ascending=False)
        self.movie_data = movies_df
        
        print(f"✅ Trained on {len(self.popular_movies)} popular movies")
        return self
    
    def recommend(self, user_id=None, n=10):
        """Get top N popular movies"""
        recommendations = self.popular_movies.head(n)[['movieId', 'title', 'genres', 'popularity_score']]
        return recommendations.to_dict('records')
    
    def explain(self, movie_id):
        """Generate explanation for recommendation"""
        movie = self.popular_movies[self.popular_movies['movieId'] == movie_id]
        if len(movie) > 0:
            movie = movie.iloc[0]
            return f"This movie is highly popular with {int(movie['rating_count'])} ratings and an average score of {movie['rating_mean']:.2f}/5.0"
        return "Popular movie among users"

# Train and save
pop_model = PopularityRecommender()
pop_model.fit(train_ratings, movies_df)

with open('models/popularity_model.pkl', 'wb') as f:
    pickle.dump(pop_model, f)

print("✅ Popularity model saved!")

# ============================================================================
# PART 7: MODEL 2 - COLLABORATIVE FILTERING (USER-USER)
# ============================================================================

print("\n👥 BUILDING USER-USER COLLABORATIVE FILTERING...")

class UserUserCF:
    def __init__(self, k=20):
        self.k = k  # Number of neighbors
        self.user_item_matrix = None
        self.user_similarity = None
        self.movies_df = None
    
    def fit(self, ratings_df, movies_df):
        """Train user-user CF model"""
        # Create user-item matrix
        self.user_item_matrix = ratings_df.pivot(
            index='userId', 
            columns='movieId', 
            values='rating'
        ).fillna(0)
        
        # Calculate user similarity (cosine)
        print("Calculating user similarity matrix...")
        user_matrix = self.user_item_matrix.values
        self.user_similarity = cosine_similarity(user_matrix)
        
        self.movies_df = movies_df
        print(f"✅ Trained on {len(self.user_item_matrix)} users")
        return self
    
    def recommend(self, user_id, n=10):
        """Get top N recommendations for user"""
        if user_id not in self.user_item_matrix.index:
            return []
        
        user_idx = self.user_item_matrix.index.get_loc(user_id)
        
        # Get similar users
        user_similarities = self.user_similarity[user_idx]
        similar_user_indices = user_similarities.argsort()[::-1][1:self.k+1]
        
        # Get movies rated by similar users
        user_ratings = self.user_item_matrix.iloc[user_idx]
        unrated_movies = user_ratings[user_ratings == 0].index
        
        # Predict ratings for unrated movies
        predictions = {}
        for movie_id in unrated_movies:
            if movie_id in self.user_item_matrix.columns:
                movie_ratings = self.user_item_matrix[movie_id].iloc[similar_user_indices]
                similarities = user_similarities[similar_user_indices]
                
                # Weighted average
                rated_mask = movie_ratings > 0
                if rated_mask.sum() > 0:
                    pred = np.sum(movie_ratings[rated_mask] * similarities[rated_mask]) / np.sum(similarities[rated_mask])
                    predictions[movie_id] = pred
        
        # Get top N
        top_movies = sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:n]
        
        # Format recommendations
        recommendations = []
        for movie_id, score in top_movies:
            movie_info = self.movies_df[self.movies_df['movieId'] == movie_id]
            if len(movie_info) > 0:
                movie_info = movie_info.iloc[0]
                recommendations.append({
                    'movieId': movie_id,
                    'title': movie_info['title'],
                    'genres': movie_info['genres'],
                    'predicted_rating': score
                })
        
        return recommendations
    
    def explain(self, user_id, movie_id):
        """Generate explanation"""
        return f"Recommended because users similar to you enjoyed this movie"

# Train and save
user_cf_model = UserUserCF(k=20)
user_cf_model.fit(train_ratings, movies_df)

with open('models/user_cf_model.pkl', 'wb') as f:
    pickle.dump(user_cf_model, f)

print("✅ User-User CF model saved!")

# ============================================================================
# PART 8: MODEL 3 - ITEM-ITEM COLLABORATIVE FILTERING
# ============================================================================

print("\n🎬 BUILDING ITEM-ITEM COLLABORATIVE FILTERING...")

class ItemItemCF:
    def __init__(self, k=20):
        self.k = k
        self.item_similarity = None
        self.user_ratings = None
        self.movies_df = None
    
    def fit(self, ratings_df, movies_df):
        """Train item-item CF model"""
        # Create item-user matrix
        item_user_matrix = ratings_df.pivot(
            index='movieId',
            columns='userId',
            values='rating'
        ).fillna(0)
        
        # Calculate item similarity
        print("Calculating item similarity matrix...")
        self.item_similarity = cosine_similarity(item_user_matrix.values)
        self.item_similarity = pd.DataFrame(
            self.item_similarity,
            index=item_user_matrix.index,
            columns=item_user_matrix.index
        )
        
        # Store user ratings for predictions
        self.user_ratings = ratings_df.groupby('userId').apply(
            lambda x: dict(zip(x['movieId'], x['rating']))
        ).to_dict()
        
        self.movies_df = movies_df
        print(f"✅ Trained on {len(item_user_matrix)} items")
        return self
    
    def recommend(self, user_id, n=10):
        """Get top N recommendations"""
        if user_id not in self.user_ratings:
            return []
        
        user_rated_movies = self.user_ratings[user_id]
        
        # Find similar items to user's rated items
        predictions = {}
        for rated_movie, rating in user_rated_movies.items():
            if rated_movie in self.item_similarity.index:
                similar_movies = self.item_similarity[rated_movie]
                
                for movie_id, similarity in similar_movies.items():
                    if movie_id not in user_rated_movies:
                        if movie_id not in predictions:
                            predictions[movie_id] = 0
                        predictions[movie_id] += similarity * rating
        
        # Normalize and get top N
        top_movies = sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:n]
        
        recommendations = []
        for movie_id, score in top_movies:
            movie_info = self.movies_df[self.movies_df['movieId'] == movie_id]
            if len(movie_info) > 0:
                movie_info = movie_info.iloc[0]
                recommendations.append({
                    'movieId': movie_id,
                    'title': movie_info['title'],
                    'genres': movie_info['genres'],
                    'predicted_rating': score
                })
        
        return recommendations
    
    def explain(self, user_id, movie_id):
        """Generate explanation"""
        if user_id in self.user_ratings:
            user_movies = list(self.user_ratings[user_id].keys())
            if movie_id in self.item_similarity.index:
                similar = self.item_similarity[movie_id][user_movies].nlargest(3)
                similar_titles = self.movies_df[self.movies_df['movieId'].isin(similar.index)]['title'].tolist()
                if similar_titles:
                    return f"Similar to movies you liked: {', '.join(similar_titles[:2])}"
        return "Based on your viewing history"

# Train and save
item_cf_model = ItemItemCF(k=20)
item_cf_model.fit(train_ratings, movies_df)

with open('models/item_cf_model.pkl', 'wb') as f:
    pickle.dump(item_cf_model, f)

print("✅ Item-Item CF model saved!")

# ============================================================================
# PART 9: MODEL 4 - MATRIX FACTORIZATION (SVD)
# ============================================================================

print("\n🧮 BUILDING MATRIX FACTORIZATION MODEL (SVD)...")

# Prepare data for Surprise library
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(train_ratings[['userId', 'movieId', 'rating']], reader)

# Train SVD
print("Training SVD model...")
svd_model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02, verbose=True)
trainset = data.build_full_trainset()
svd_model.fit(trainset)

# Save model
with open('models/svd_model.pkl', 'wb') as f:
    pickle.dump(svd_model, f)

print("✅ SVD model saved!")

# Evaluation on train set
from surprise import accuracy

predictions = svd_model.test(trainset.build_testset())
print(f"\nTrain RMSE: {accuracy.rmse(predictions):.4f}")
print(f"Train MAE: {accuracy.mae(predictions):.4f}")

# ============================================================================
# PART 10: MODEL 5 - CONTENT-BASED FILTERING
# ============================================================================

print("\n📝 BUILDING CONTENT-BASED MODEL...")

class ContentBasedRecommender:
    def __init__(self):
        self.tfidf_matrix = None
        self.movie_indices = None
        self.movies_df = None
        self.user_profiles = {}
    
    def fit(self, ratings_df, movies_df):
        """Train content-based model"""
        # Create TF-IDF matrix
        tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
        self.tfidf_matrix = tfidf.fit_transform(movies_df['content_features'])
        self.movie_indices = movies_df['movieId'].values
        self.movies_df = movies_df
        
        # Build user profiles based on rated movies
        print("Building user profiles...")
        for user_id in ratings_df['userId'].unique():
            user_ratings = ratings_df[ratings_df['userId'] == user_id]
            
            # Get movies rated highly (>= 4.0)
            liked_movies = user_ratings[user_ratings['rating'] >= 4.0]['movieId'].values
            
            # Build profile as average of liked movie features
            if len(liked_movies) > 0:
                liked_indices = [np.where(self.movie_indices == mid)[0][0] 
                               for mid in liked_movies if mid in self.movie_indices]
                
                if liked_indices:
                    user_profile = np.array(self.tfidf_matrix[liked_indices].mean(axis=0)).flatten()
                    self.user_profiles[user_id] = user_profile
        
        print(f"✅ Created profiles for {len(self.user_profiles)} users")
        return self
    
    def recommend(self, user_id, n=10):
        """Get top N content-based recommendations"""
        if user_id not in self.user_profiles:
            return []
        
        user_profile = self.user_profiles[user_id]
        
        # Calculate similarity to all movies
        similarities = cosine_similarity([user_profile], self.tfidf_matrix)[0]
        
        # Get top similar movies
        top_indices = similarities.argsort()[::-1][:n*2]  # Get more for filtering
        
        recommendations = []
        for idx in top_indices:
            if len(recommendations) >= n:
                break
            
            movie_id = self.movie_indices[idx]
            movie_info = self.movies_df[self.movies_df['movieId'] == movie_id].iloc[0]
            
            recommendations.append({
                'movieId': movie_id,
                'title': movie_info['title'],
                'genres': movie_info['genres'],
                'similarity_score': similarities[idx]
            })
        
        return recommendations
    
    def explain(self, user_id, movie_id):
        """Generate explanation"""
        movie = self.movies_df[self.movies_df['movieId'] == movie_id]
        if len(movie) > 0:
            genres = movie.iloc[0]['genres']
            return f"Matches your interest in genres: {genres}"
        return "Based on content similarity to your preferences"

# Train and save
content_model = ContentBasedRecommender()
content_model.fit(train_ratings, movies_df)

with open('models/content_model.pkl', 'wb') as f:
    pickle.dump(content_model, f)

print("✅ Content-based model saved!")

# ============================================================================
# PART 11: HYBRID RECOMMENDER
# ============================================================================

print("\n🎯 BUILDING HYBRID RECOMMENDER...")

class HybridRecommender:
    def __init__(self, models, weights=None):
        """
        models: dict of {name: model}
        weights: dict of {name: weight} (defaults to equal weights)
        """
        self.models = models
        self.weights = weights or {name: 1.0/len(models) for name in models}
        self.movies_df = None
    
    def fit(self, movies_df):
        self.movies_df = movies_df
        return self
    
    def recommend(self, user_id, n=10):
        """Get hybrid recommendations"""
        all_recommendations = {}
        
        # Get recommendations from each model
        for name, model in self.models.items():
            try:
                recs = model.recommend(user_id, n=n*2)
                weight = self.weights[name]
                
                for rec in recs:
                    movie_id = rec['movieId']
                    score = rec.get('predicted_rating', rec.get('popularity_score', rec.get('similarity_score', 1.0)))
                    
                    if movie_id not in all_recommendations:
                        all_recommendations[movie_id] = {
                            'score': 0,
                            'count': 0,
                            'sources': []
                        }
                    
                    all_recommendations[movie_id]['score'] += score * weight
                    all_recommendations[movie_id]['count'] += 1
                    all_recommendations[movie_id]['sources'].append(name)
            except Exception as e:
                print(f"Warning: {name} model failed: {e}")
                continue
        
        # Normalize scores and get top N
        for movie_id in all_recommendations:
            all_recommendations[movie_id]['final_score'] = (
                all_recommendations[movie_id]['score'] / all_recommendations[movie_id]['count']
            )
        
        top_movies = sorted(
            all_recommendations.items(),
            key=lambda x: x[1]['final_score'],
            reverse=True
        )[:n]
        
        # Format recommendations
        recommendations = []
        for movie_id, data in top_movies:
            movie_info = self.movies_df[self.movies_df['movieId'] == movie_id]
            if len(movie_info) > 0:
                movie_info = movie_info.iloc[0]
                recommendations.append({
                    'movieId': movie_id,
                    'title': movie_info['title'],
                    'genres': movie_info['genres'],
                    'hybrid_score': data['final_score'],
                    'sources': data['sources']
                })
        
        return recommendations
    
    def explain(self, user_id, movie_id, sources):
        """Generate hybrid explanation"""
        explanations = []
        for source in sources:
            if source in self.models:
                exp = self.models[source].explain(user_id, movie_id)
                explanations.append(f"{source}: {exp}")
        
        return " | ".join(explanations)

# Create hybrid model
hybrid_models = {
    'user_cf': user_cf_model,
    'item_cf': item_cf_model,
    'content': content_model,
}

hybrid_weights = {
    'user_cf': 0.4,
    'item_cf': 0.4,
    'content': 0.2
}

hybrid_model = HybridRecommender(hybrid_models, hybrid_weights)
hybrid_model.fit(movies_df)

with open('models/hybrid_model.pkl', 'wb') as f:
    pickle.dump(hybrid_model, f)

print("✅ Hybrid model saved!")

# ============================================================================
# PART 12: EVALUATION METRICS
# ============================================================================

print("\n📈 CALCULATING EVALUATION METRICS...")

def precision_at_k(recommended, relevant, k=10):
    """Precision@K"""
    recommended_k = recommended[:k]
    relevant_set = set(relevant)
    hits = len([r for r in recommended_k if r in relevant_set])
    return hits / k if k > 0 else 0

def recall_at_k(recommended, relevant, k=10):
    """Recall@K"""
    recommended_k = recommended[:k]
    relevant_set = set(relevant)
    hits = len([r for r in recommended_k if r in relevant_set])
    return hits / len(relevant_set) if len(relevant_set) > 0 else 0

def ndcg_at_k(recommended, relevant, k=10):
    """Normalized Discounted Cumulative Gain@K"""
    recommended_k = recommended[:k]
    relevant_set = set(relevant)
    
    dcg = sum([1.0 / np.log2(idx + 2) if rec in relevant_set else 0 
               for idx, rec in enumerate(recommended_k)])
    
    idcg = sum([1.0 / np.log2(idx + 2) for idx in range(min(len(relevant), k))])
    
    return dcg / idcg if idcg > 0 else 0

def evaluate_model(model, test_ratings, k=10):
    """Evaluate a recommendation model"""
    precisions = []
    recalls = []
    ndcgs = []
    
    # Sample users for faster evaluation
    test_users = test_ratings['userId'].unique()[:100]  # Sample 100 users
    
    for user_id in test_users:
        # Get user's test items (relevant items)
        relevant_items = test_ratings[test_ratings['userId'] == user_id]['movieId'].tolist()
        
        if len(relevant_items) == 0:
            continue
        
        try:
            # Get recommendations
            recs = model.recommend(user_id, n=k)
            recommended_items = [r['movieId'] for r in recs]
            
            if len(recommended_items) > 0:
                precisions.append(precision_at_k(recommended_items, relevant_items, k))
                recalls.append(recall_at_k(recommended_items, relevant_items, k))
                ndcgs.append(ndcg_at_k(recommended_items, relevant_items, k))
        except:
            continue
    
    return {
        'Precision@K': np.mean(precisions) if precisions else 0,
        'Recall@K': np.mean(recalls) if recalls else 0,
        'NDCG@K': np.mean(ndcgs) if ndcgs else 0
    }

# Evaluate models
print("\n🎯 Evaluating models on test set...")

results = {}

print("\n1. Evaluating Hybrid Model...")
results['Hybrid'] = evaluate_model(hybrid_model, test_ratings, k=10)

print("\n2. Evaluating User-User CF...")
results['User-User CF'] = evaluate_model(user_cf_model, test_ratings, k=10)

print("\n3. Evaluating Item-Item CF...")
results['Item-Item CF'] = evaluate_model(item_cf_model, test_ratings, k=10)

print("\n4. Evaluating Content-Based...")
results['Content-Based'] = evaluate_model(content_model, test_ratings, k=10)

# Display results
print("\n" + "="*70)
print("EVALUATION RESULTS (K=10)")
print("="*70)

results_df = pd.DataFrame(results).T
print(results_df.to_string())

# Save results
results_df.to_csv('evaluation_results.csv')
print("\n✅ Results saved to 'evaluation_results.csv'")

# ============================================================================
# PART 13: DIVERSITY & NOVELTY METRICS
# ============================================================================

print("\n🌈 CALCULATING DIVERSITY METRICS...")

def calculate_diversity_metrics(model, test_users, movies_df, k=10):
    """Calculate diversity and novelty metrics"""
    
    # Catalog coverage
    all_recommended = set()
    total_catalog = len(movies_df)
    
    # Intra-list diversity
    intra_diversities = []
    
    for user_id in test_users[:50]:  # Sample for speed
        try:
            recs = model.recommend(user_id, n=k)
            rec_ids = [r['movieId'] for r in recs]
            
            all_recommended.update(rec_ids)
            
            # Calculate genre diversity
            rec_genres = []
            for movie_id in rec_ids:
                movie = movies_df[movies_df['movieId'] == movie_id]
                if len(movie) > 0:
                    genres = movie.iloc[0]['genres'].split('|')
                    rec_genres.extend(genres)
            
            # Unique genres / total items
            if len(rec_ids) > 0:
                diversity = len(set(rec_genres)) / len(rec_ids)
                intra_diversities.append(diversity)
        except:
            continue
    
    catalog_coverage = len(all_recommended) / total_catalog
    avg_intra_diversity = np.mean(intra_diversities) if intra_diversities else 0
    
    return {
        'Catalog Coverage': catalog_coverage,
        'Intra-List Diversity': avg_intra_diversity
    }

# Calculate for hybrid model
test_users = test_ratings['userId'].unique()
diversity_metrics = calculate_diversity_metrics(hybrid_model, test_users, movies_df, k=10)

print("\n" + "="*70)
print("DIVERSITY METRICS (Hybrid Model)")
print("="*70)
for metric, value in diversity_metrics.items():
    print(f"{metric}: {value:.4f}")

# Save diversity metrics
with open('diversity_metrics.json', 'w') as f:
    json.dump(diversity_metrics, f, indent=2)

print("\n✅ Diversity metrics saved to 'diversity_metrics.json'")

# ============================================================================
# PART 14: EXPLAINABILITY EXAMPLES
# ============================================================================

print("\n💡 GENERATING EXPLANATION EXAMPLES...")

# Get a sample user
sample_user = train_ratings['userId'].iloc[0]

print(f"\n📌 Sample Recommendations for User {sample_user}:")
print("="*70)

# Get hybrid recommendations
hybrid_recs = hybrid_model.recommend(sample_user, n=5)

for i, rec in enumerate(hybrid_recs, 1):
    print(f"\n{i}. {rec['title']}")
    print(f"   Genres: {rec['genres']}")
    print(f"   Score: {rec['hybrid_score']:.4f}")
    print(f"   Sources: {', '.join(rec['sources'])}")
    
    # Generate explanation
    explanation = hybrid_model.explain(sample_user, rec['movieId'], rec['sources'])
    print(f"   📖 Why: {explanation}")

# ============================================================================
# PART 15: SAVE MODEL METADATA
# ============================================================================

print("\n💾 SAVING MODEL METADATA...")

metadata = {
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset_stats': {
        'total_ratings': len(ratings_df),
        'total_users': ratings_df['userId'].nunique(),
        'total_movies': ratings_df['movieId'].nunique(),
        'train_size': len(train_ratings),
        'test_size': len(test_ratings)
    },
    'models': {
        'popularity': 'PopularityRecommender',
        'user_cf': 'UserUserCF(k=20)',
        'item_cf': 'ItemItemCF(k=20)',
        'svd': 'SVD(factors=100)',
        'content': 'ContentBasedRecommender',
        'hybrid': 'HybridRecommender(weights=user_cf:0.4, item_cf:0.4, content:0.2)'
    },
    'evaluation': results,
    'diversity': diversity_metrics
}

with open('models/model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✅ Metadata saved to 'models/model_metadata.json'")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*70)
print("🎉 REELSENSE TRAINING PIPELINE COMPLETE!")
print("="*70)
print("\n📁 Files Generated:")
print("  ✓ models/popularity_model.pkl")
print("  ✓ models/user_cf_model.pkl")
print("  ✓ models/item_cf_model.pkl")
print("  ✓ models/svd_model.pkl")
print("  ✓ models/content_model.pkl")
print("  ✓ models/hybrid_model.pkl")
print("  ✓ models/model_metadata.json")
print("  ✓ evaluation_results.csv")
print("  ✓ diversity_metrics.json")
print("  ✓ eda_plots/ (6 visualization files)")

print("\n📊 Best Model Performance:")
best_model = max(results.items(), key=lambda x: x[1]['NDCG@K'])
print(f"  {best_model[0]}: NDCG@10 = {best_model[1]['NDCG@K']:.4f}")

print("\n✨ Next Step: Deploy with Streamlit UI!")
print("="*70)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.12/dist-package

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

In [1]:
"""
ReelSense: Explainable Movie Recommender System
Kaggle Training Pipeline - FIXED VERSION
"""

# ============================================================================
# PART 0: FIX NUMPY COMPATIBILITY
# ============================================================================

# import subprocess
# import sys

# print("🔧 Checking NumPy compatibility...")
# try:
#     import numpy as np
#     if int(np.__version__.split('.')[0]) >= 2:
#         print("⚠️ NumPy 2.x detected. Downgrading to NumPy 1.x for surprise library compatibility...")
#         subprocess.check_call([sys.executable, "-m", "pip", "install", "numpy<2", "-q"])
#         print("✅ NumPy downgraded. Please RESTART the kernel and run again.")
#         print("   In Kaggle: Click 'Session' -> 'Restart Session'")
#         print("   In Colab: Click 'Runtime' -> 'Restart runtime'")
#         sys.exit(0)
#     else:
#         print(f"✅ NumPy {np.__version__} is compatible!")
# except:
#     pass

# ============================================================================
# PART 1: IMPORTS AND SETUP
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

# Surprise library for Matrix Factorization
try:
    from surprise import SVD, Dataset, Reader
    from surprise.model_selection import cross_validate
    SURPRISE_AVAILABLE = True
    print("✅ Surprise library loaded successfully!")
except ImportError as e:
    print(f"⚠️ Warning: Surprise library not available: {e}")
    print("   SVD model will be skipped. Other models will work fine.")
    SURPRISE_AVAILABLE = False

# For saving models
import pickle
import json
import os

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All libraries imported successfully!")

# ============================================================================
# PART 2: DATA LOADING
# ============================================================================

# Load datasets
ratings_df = pd.read_csv('/kaggle/input/movie-recommendation/ratings.csv')
movies_df = pd.read_csv('/kaggle/input/movie-recommendation/movies.csv')
tags_df = pd.read_csv('/kaggle/input/movie-recommendation/tags.csv')
links_df = pd.read_csv('/kaggle/input/movie-recommendation/links.csv')

print("📊 Dataset Statistics:")
print(f"Ratings: {len(ratings_df):,} rows")
print(f"Movies: {len(movies_df):,} rows")
print(f"Tags: {len(tags_df):,} rows")
print(f"Links: {len(links_df):,} rows")
print(f"\nUsers: {ratings_df['userId'].nunique():,}")
print(f"Movies rated: {ratings_df['movieId'].nunique():,}")

# Display samples
print("\n📋 Sample Data:")
print("\nRatings:")
print(ratings_df.head())
print("\nMovies:")
print(movies_df.head())
print("\nTags:")
print(tags_df.head())

# ============================================================================
# PART 3: DATA PREPROCESSING
# ============================================================================

print("\n🧹 PREPROCESSING DATA...")

# Convert timestamp to datetime
ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'], unit='s')
if 'timestamp' in tags_df.columns:
    tags_df['timestamp'] = pd.to_datetime(tags_df['timestamp'], unit='s')

# Sort by timestamp
ratings_df = ratings_df.sort_values('timestamp')

# Clean genres
movies_df['genres_list'] = movies_df['genres'].str.split('|')
movies_df['genres_clean'] = movies_df['genres'].str.replace('|', ' ')

# Clean tags
if len(tags_df) > 0:
    tags_df['tag'] = tags_df['tag'].str.lower().str.strip()
    
    # Aggregate tags per movie
    movie_tags = tags_df.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()
    movie_tags.columns = ['movieId', 'tags_combined']
    
    # Merge with movies
    movies_df = movies_df.merge(movie_tags, on='movieId', how='left')
    movies_df['tags_combined'] = movies_df['tags_combined'].fillna('')
else:
    movies_df['tags_combined'] = ''

# Create content features (genres + tags)
movies_df['content_features'] = movies_df['genres_clean'] + ' ' + movies_df['tags_combined']

# Extract year from title
movies_df['year'] = movies_df['title'].str.extract(r'\((\d{4})\)').astype(float)

print("✅ Preprocessing complete!")

# ============================================================================
# PART 4: EXPLORATORY DATA ANALYSIS
# ============================================================================

print("\n📊 PERFORMING EDA...")

# Create EDA directory
os.makedirs('eda_plots', exist_ok=True)

# 1. Rating Distribution
plt.figure(figsize=(10, 6))
ratings_df['rating'].value_counts().sort_index().plot(kind='bar', color='steelblue')
plt.title('Distribution of Ratings', fontsize=16, fontweight='bold')
plt.xlabel('Rating', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig('eda_plots/rating_distribution.png', dpi=300, bbox_inches='tight')
plt.close()

# 2. User Activity Distribution
plt.figure(figsize=(12, 6))
user_activity = ratings_df['userId'].value_counts()
plt.hist(user_activity, bins=50, color='coral', edgecolor='black')
plt.title('User Activity Distribution', fontsize=16, fontweight='bold')
plt.xlabel('Number of Ratings per User', fontsize=12)
plt.ylabel('Number of Users', fontsize=12)
plt.axvline(user_activity.mean(), color='red', linestyle='--', label=f'Mean: {user_activity.mean():.1f}')
plt.legend()
plt.tight_layout()
plt.savefig('eda_plots/user_activity.png', dpi=300, bbox_inches='tight')
plt.close()

# 3. Movie Popularity (Long-tail)
plt.figure(figsize=(12, 6))
movie_counts = ratings_df['movieId'].value_counts()
plt.plot(range(len(movie_counts)), sorted(movie_counts.values, reverse=True), color='darkgreen')
plt.title('Long-tail Distribution of Movie Popularity', fontsize=16, fontweight='bold')
plt.xlabel('Movie Rank', fontsize=12)
plt.ylabel('Number of Ratings', fontsize=12)
plt.yscale('log')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('eda_plots/movie_longtail.png', dpi=300, bbox_inches='tight')
plt.close()

# 4. Genre Popularity
all_genres = []
for genres_list in movies_df['genres_list']:
    if isinstance(genres_list, list):
        all_genres.extend(genres_list)

genre_counts = pd.Series(all_genres).value_counts().head(15)

plt.figure(figsize=(12, 6))
genre_counts.plot(kind='barh', color='purple')
plt.title('Top 15 Most Popular Genres', fontsize=16, fontweight='bold')
plt.xlabel('Count', fontsize=12)
plt.ylabel('Genre', fontsize=12)
plt.tight_layout()
plt.savefig('eda_plots/genre_popularity.png', dpi=300, bbox_inches='tight')
plt.close()

# 5. Average Rating by Genre
genre_ratings = {}
for idx, row in movies_df.iterrows():
    if isinstance(row['genres_list'], list):
        movie_ratings = ratings_df[ratings_df['movieId'] == row['movieId']]['rating']
        if len(movie_ratings) > 0:
            avg_rating = movie_ratings.mean()
            for genre in row['genres_list']:
                if genre not in genre_ratings:
                    genre_ratings[genre] = []
                genre_ratings[genre].append(avg_rating)

genre_avg = {k: np.mean(v) for k, v in genre_ratings.items() if len(v) > 10}
genre_avg_series = pd.Series(genre_avg).sort_values(ascending=False)

plt.figure(figsize=(12, 6))
genre_avg_series.plot(kind='barh', color='teal')
plt.title('Average Rating by Genre', fontsize=16, fontweight='bold')
plt.xlabel('Average Rating', fontsize=12)
plt.ylabel('Genre', fontsize=12)
plt.tight_layout()
plt.savefig('eda_plots/genre_avg_rating.png', dpi=300, bbox_inches='tight')
plt.close()

# 6. Rating Trends Over Time
ratings_df['year_rated'] = ratings_df['timestamp'].dt.year
yearly_stats = ratings_df.groupby('year_rated').agg({
    'rating': ['mean', 'count']
}).reset_index()
yearly_stats.columns = ['year', 'avg_rating', 'count']

fig, ax1 = plt.subplots(figsize=(12, 6))
ax2 = ax1.twinx()

ax1.plot(yearly_stats['year'], yearly_stats['avg_rating'], 'b-o', label='Avg Rating')
ax2.bar(yearly_stats['year'], yearly_stats['count'], alpha=0.3, color='orange', label='Count')

ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Average Rating', fontsize=12, color='b')
ax2.set_ylabel('Number of Ratings', fontsize=12, color='orange')
ax1.set_title('Rating Trends Over Time', fontsize=16, fontweight='bold')
ax1.tick_params(axis='y', labelcolor='b')
ax2.tick_params(axis='y', labelcolor='orange')
plt.tight_layout()
plt.savefig('eda_plots/rating_trends.png', dpi=300, bbox_inches='tight')
plt.close()

print("✅ EDA plots saved to 'eda_plots/' directory")

# ============================================================================
# PART 5: TRAIN-TEST SPLIT (LEAVE-LAST-N)
# ============================================================================

print("\n🔀 CREATING TRAIN-TEST SPLIT...")

# Leave-last-1 per user
def leave_last_n_split(df, n=1):
    """Split data leaving last N ratings per user for testing"""
    train_list = []
    test_list = []
    
    for user_id in df['userId'].unique():
        user_data = df[df['userId'] == user_id].sort_values('timestamp')
        
        if len(user_data) > n:
            train_list.append(user_data.iloc[:-n])
            test_list.append(user_data.iloc[-n:])
        else:
            train_list.append(user_data)
    
    train_df = pd.concat(train_list, ignore_index=True)
    test_df = pd.concat(test_list, ignore_index=True)
    
    return train_df, test_df

train_ratings, test_ratings = leave_last_n_split(ratings_df, n=1)

print(f"Train set: {len(train_ratings):,} ratings")
print(f"Test set: {len(test_ratings):,} ratings")
print(f"Train users: {train_ratings['userId'].nunique()}")
print(f"Test users: {test_ratings['userId'].nunique()}")

# Create models directory
os.makedirs('models', exist_ok=True)

# ============================================================================
# PART 6: MODEL 1 - POPULARITY-BASED RECOMMENDER
# ============================================================================

print("\n🌟 BUILDING POPULARITY-BASED MODEL...")

class PopularityRecommender:
    def __init__(self):
        self.popular_movies = None
        self.movie_data = None
    
    def fit(self, ratings_df, movies_df):
        """Train popularity model"""
        movie_stats = ratings_df.groupby('movieId').agg({
            'rating': ['count', 'mean']
        }).reset_index()
        movie_stats.columns = ['movieId', 'rating_count', 'rating_mean']
        
        # Filter movies with at least 50 ratings
        movie_stats = movie_stats[movie_stats['rating_count'] >= 50]
        
        # Calculate popularity score (weighted rating)
        C = movie_stats['rating_mean'].mean()
        m = movie_stats['rating_count'].quantile(0.75)
        
        movie_stats['popularity_score'] = (
            (movie_stats['rating_count'] / (movie_stats['rating_count'] + m)) * movie_stats['rating_mean'] +
            (m / (movie_stats['rating_count'] + m)) * C
        )
        
        # Merge with movie data
        self.popular_movies = movie_stats.merge(movies_df, on='movieId')
        self.popular_movies = self.popular_movies.sort_values('popularity_score', ascending=False)
        self.movie_data = movies_df
        
        print(f"✅ Trained on {len(self.popular_movies)} popular movies")
        return self
    
    def recommend(self, user_id=None, n=10):
        """Get top N popular movies"""
        recommendations = self.popular_movies.head(n)[['movieId', 'title', 'genres', 'popularity_score']]
        return recommendations.to_dict('records')
    
    def explain(self, user_id, movie_id):
        """Generate explanation for recommendation"""
        movie = self.popular_movies[self.popular_movies['movieId'] == movie_id]
        if len(movie) > 0:
            movie = movie.iloc[0]
            return f"This movie is highly popular with {int(movie['rating_count'])} ratings and an average score of {movie['rating_mean']:.2f}/5.0"
        return "Popular movie among users"

# Train and save
pop_model = PopularityRecommender()
pop_model.fit(train_ratings, movies_df)

with open('models/popularity_model.pkl', 'wb') as f:
    pickle.dump(pop_model, f)

print("✅ Popularity model saved!")

# ============================================================================
# PART 7: MODEL 2 - COLLABORATIVE FILTERING (USER-USER)
# ============================================================================

print("\n👥 BUILDING USER-USER COLLABORATIVE FILTERING...")

class UserUserCF:
    def __init__(self, k=20):
        self.k = k  # Number of neighbors
        self.user_item_matrix = None
        self.user_similarity = None
        self.movies_df = None
    
    def fit(self, ratings_df, movies_df):
        """Train user-user CF model"""
        # Create user-item matrix
        self.user_item_matrix = ratings_df.pivot(
            index='userId', 
            columns='movieId', 
            values='rating'
        ).fillna(0)
        
        # Calculate user similarity (cosine)
        print("Calculating user similarity matrix...")
        user_matrix = self.user_item_matrix.values
        self.user_similarity = cosine_similarity(user_matrix)
        
        self.movies_df = movies_df
        print(f"✅ Trained on {len(self.user_item_matrix)} users")
        return self
    
    def recommend(self, user_id, n=10):
        """Get top N recommendations for user"""
        if user_id not in self.user_item_matrix.index:
            return []
        
        user_idx = self.user_item_matrix.index.get_loc(user_id)
        
        # Get similar users
        user_similarities = self.user_similarity[user_idx]
        similar_user_indices = user_similarities.argsort()[::-1][1:self.k+1]
        
        # Get movies rated by similar users
        user_ratings = self.user_item_matrix.iloc[user_idx]
        unrated_movies = user_ratings[user_ratings == 0].index
        
        # Predict ratings for unrated movies
        predictions = {}
        for movie_id in unrated_movies:
            if movie_id in self.user_item_matrix.columns:
                movie_ratings = self.user_item_matrix[movie_id].iloc[similar_user_indices]
                similarities = user_similarities[similar_user_indices]
                
                # Weighted average
                rated_mask = movie_ratings > 0
                if rated_mask.sum() > 0:
                    pred = np.sum(movie_ratings[rated_mask] * similarities[rated_mask]) / np.sum(similarities[rated_mask])
                    predictions[movie_id] = pred
        
        # Get top N
        top_movies = sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:n]
        
        # Format recommendations
        recommendations = []
        for movie_id, score in top_movies:
            movie_info = self.movies_df[self.movies_df['movieId'] == movie_id]
            if len(movie_info) > 0:
                movie_info = movie_info.iloc[0]
                recommendations.append({
                    'movieId': movie_id,
                    'title': movie_info['title'],
                    'genres': movie_info['genres'],
                    'predicted_rating': score
                })
        
        return recommendations
    
    def explain(self, user_id, movie_id):
        """Generate explanation"""
        return f"Recommended because users similar to you enjoyed this movie"

# Train and save
user_cf_model = UserUserCF(k=20)
user_cf_model.fit(train_ratings, movies_df)

with open('models/user_cf_model.pkl', 'wb') as f:
    pickle.dump(user_cf_model, f)

print("✅ User-User CF model saved!")

# ============================================================================
# PART 8: MODEL 3 - ITEM-ITEM COLLABORATIVE FILTERING
# ============================================================================

print("\n🎬 BUILDING ITEM-ITEM COLLABORATIVE FILTERING...")

class ItemItemCF:
    def __init__(self, k=20):
        self.k = k
        self.item_similarity = None
        self.user_ratings = None
        self.movies_df = None
    
    def fit(self, ratings_df, movies_df):
        """Train item-item CF model"""
        # Create item-user matrix
        item_user_matrix = ratings_df.pivot(
            index='movieId',
            columns='userId',
            values='rating'
        ).fillna(0)
        
        # Calculate item similarity
        print("Calculating item similarity matrix...")
        self.item_similarity = cosine_similarity(item_user_matrix.values)
        self.item_similarity = pd.DataFrame(
            self.item_similarity,
            index=item_user_matrix.index,
            columns=item_user_matrix.index
        )
        
        # Store user ratings for predictions
        self.user_ratings = ratings_df.groupby('userId').apply(
            lambda x: dict(zip(x['movieId'], x['rating']))
        ).to_dict()
        
        self.movies_df = movies_df
        print(f"✅ Trained on {len(item_user_matrix)} items")
        return self
    
    def recommend(self, user_id, n=10):
        """Get top N recommendations"""
        if user_id not in self.user_ratings:
            return []
        
        user_rated_movies = self.user_ratings[user_id]
        
        # Find similar items to user's rated items
        predictions = {}
        for rated_movie, rating in user_rated_movies.items():
            if rated_movie in self.item_similarity.index:
                similar_movies = self.item_similarity[rated_movie]
                
                for movie_id, similarity in similar_movies.items():
                    if movie_id not in user_rated_movies:
                        if movie_id not in predictions:
                            predictions[movie_id] = 0
                        predictions[movie_id] += similarity * rating
        
        # Normalize and get top N
        top_movies = sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:n]
        
        recommendations = []
        for movie_id, score in top_movies:
            movie_info = self.movies_df[self.movies_df['movieId'] == movie_id]
            if len(movie_info) > 0:
                movie_info = movie_info.iloc[0]
                recommendations.append({
                    'movieId': movie_id,
                    'title': movie_info['title'],
                    'genres': movie_info['genres'],
                    'predicted_rating': score
                })
        
        return recommendations
    
    def explain(self, user_id, movie_id):
        """Generate explanation"""
        if user_id in self.user_ratings:
            user_movies = list(self.user_ratings[user_id].keys())
            if movie_id in self.item_similarity.index:
                similar = self.item_similarity[movie_id][user_movies].nlargest(3)
                similar_titles = self.movies_df[self.movies_df['movieId'].isin(similar.index)]['title'].tolist()
                if similar_titles:
                    return f"Similar to movies you liked: {', '.join(similar_titles[:2])}"
        return "Based on your viewing history"

# Train and save
item_cf_model = ItemItemCF(k=20)
item_cf_model.fit(train_ratings, movies_df)

with open('models/item_cf_model.pkl', 'wb') as f:
    pickle.dump(item_cf_model, f)

print("✅ Item-Item CF model saved!")

# ============================================================================
# PART 9: MODEL 4 - MATRIX FACTORIZATION (SVD)
# ============================================================================

if SURPRISE_AVAILABLE:
    print("\n🧮 BUILDING MATRIX FACTORIZATION MODEL (SVD)...")

    # Prepare data for Surprise library
    reader = Reader(rating_scale=(0.5, 5.0))
    data = Dataset.load_from_df(train_ratings[['userId', 'movieId', 'rating']], reader)

    # Train SVD
    print("Training SVD model...")
    svd_model = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02, verbose=True)
    trainset = data.build_full_trainset()
    svd_model.fit(trainset)

    # Save model
    with open('models/svd_model.pkl', 'wb') as f:
        pickle.dump(svd_model, f)

    print("✅ SVD model saved!")

    # Evaluation on train set
    from surprise import accuracy

    predictions = svd_model.test(trainset.build_testset())
    print(f"\nTrain RMSE: {accuracy.rmse(predictions):.4f}")
    print(f"Train MAE: {accuracy.mae(predictions):.4f}")
else:
    print("\n⚠️ SKIPPING SVD MODEL (Surprise library not available)")
    svd_model = None

# ============================================================================
# PART 10: MODEL 5 - CONTENT-BASED FILTERING
# ============================================================================

print("\n📝 BUILDING CONTENT-BASED MODEL...")

class ContentBasedRecommender:
    def __init__(self):
        self.tfidf_matrix = None
        self.movie_indices = None
        self.movies_df = None
        self.user_profiles = {}
    
    def fit(self, ratings_df, movies_df):
        """Train content-based model"""
        # Create TF-IDF matrix
        tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
        self.tfidf_matrix = tfidf.fit_transform(movies_df['content_features'])
        self.movie_indices = movies_df['movieId'].values
        self.movies_df = movies_df
        
        # Build user profiles based on rated movies
        print("Building user profiles...")
        for user_id in ratings_df['userId'].unique():
            user_ratings = ratings_df[ratings_df['userId'] == user_id]
            
            # Get movies rated highly (>= 4.0)
            liked_movies = user_ratings[user_ratings['rating'] >= 4.0]['movieId'].values
            
            # Build profile as average of liked movie features
            if len(liked_movies) > 0:
                liked_indices = [np.where(self.movie_indices == mid)[0][0] 
                               for mid in liked_movies if mid in self.movie_indices]
                
                if liked_indices:
                    user_profile = np.array(self.tfidf_matrix[liked_indices].mean(axis=0)).flatten()
                    self.user_profiles[user_id] = user_profile
        
        print(f"✅ Created profiles for {len(self.user_profiles)} users")
        return self
    
    def recommend(self, user_id, n=10):
        """Get top N content-based recommendations"""
        if user_id not in self.user_profiles:
            return []
        
        user_profile = self.user_profiles[user_id]
        
        # Calculate similarity to all movies
        similarities = cosine_similarity([user_profile], self.tfidf_matrix)[0]
        
        # Get top similar movies
        top_indices = similarities.argsort()[::-1][:n*2]  # Get more for filtering
        
        recommendations = []
        for idx in top_indices:
            if len(recommendations) >= n:
                break
            
            movie_id = self.movie_indices[idx]
            movie_info = self.movies_df[self.movies_df['movieId'] == movie_id].iloc[0]
            
            recommendations.append({
                'movieId': movie_id,
                'title': movie_info['title'],
                'genres': movie_info['genres'],
                'similarity_score': similarities[idx]
            })
        
        return recommendations
    
    def explain(self, user_id, movie_id):
        """Generate explanation"""
        movie = self.movies_df[self.movies_df['movieId'] == movie_id]
        if len(movie) > 0:
            genres = movie.iloc[0]['genres']
            return f"Matches your interest in genres: {genres}"
        return "Based on content similarity to your preferences"

# Train and save
content_model = ContentBasedRecommender()
content_model.fit(train_ratings, movies_df)

with open('models/content_model.pkl', 'wb') as f:
    pickle.dump(content_model, f)

print("✅ Content-based model saved!")

# ============================================================================
# PART 11: HYBRID RECOMMENDER
# ============================================================================

print("\n🎯 BUILDING HYBRID RECOMMENDER...")

class HybridRecommender:
    def __init__(self, models, weights=None):
        """
        models: dict of {name: model}
        weights: dict of {name: weight} (defaults to equal weights)
        """
        self.models = models
        self.weights = weights or {name: 1.0/len(models) for name in models}
        self.movies_df = None
    
    def fit(self, movies_df):
        self.movies_df = movies_df
        return self
    
    def recommend(self, user_id, n=10):
        """Get hybrid recommendations"""
        all_recommendations = {}
        
        # Get recommendations from each model
        for name, model in self.models.items():
            try:
                recs = model.recommend(user_id, n=n*2)
                weight = self.weights[name]
                
                for rec in recs:
                    movie_id = rec['movieId']
                    score = rec.get('predicted_rating', rec.get('popularity_score', rec.get('similarity_score', 1.0)))
                    
                    if movie_id not in all_recommendations:
                        all_recommendations[movie_id] = {
                            'score': 0,
                            'count': 0,
                            'sources': []
                        }
                    
                    all_recommendations[movie_id]['score'] += score * weight
                    all_recommendations[movie_id]['count'] += 1
                    all_recommendations[movie_id]['sources'].append(name)
            except Exception as e:
                print(f"Warning: {name} model failed: {e}")
                continue
        
        # Normalize scores and get top N
        for movie_id in all_recommendations:
            all_recommendations[movie_id]['final_score'] = (
                all_recommendations[movie_id]['score'] / all_recommendations[movie_id]['count']
            )
        
        top_movies = sorted(
            all_recommendations.items(),
            key=lambda x: x[1]['final_score'],
            reverse=True
        )[:n]
        
        # Format recommendations
        recommendations = []
        for movie_id, data in top_movies:
            movie_info = self.movies_df[self.movies_df['movieId'] == movie_id]
            if len(movie_info) > 0:
                movie_info = movie_info.iloc[0]
                recommendations.append({
                    'movieId': movie_id,
                    'title': movie_info['title'],
                    'genres': movie_info['genres'],
                    'hybrid_score': data['final_score'],
                    'sources': data['sources']
                })
        
        return recommendations
    
    def explain(self, user_id, movie_id, sources):
        """Generate hybrid explanation"""
        explanations = []
        for source in sources:
            if source in self.models:
                exp = self.models[source].explain(user_id, movie_id)
                explanations.append(f"{source}: {exp}")
        
        return " | ".join(explanations)

# Create hybrid model
hybrid_models = {
    'user_cf': user_cf_model,
    'item_cf': item_cf_model,
    'content': content_model,
}

hybrid_weights = {
    'user_cf': 0.4,
    'item_cf': 0.4,
    'content': 0.2
}

hybrid_model = HybridRecommender(hybrid_models, hybrid_weights)
hybrid_model.fit(movies_df)

with open('models/hybrid_model.pkl', 'wb') as f:
    pickle.dump(hybrid_model, f)

print("✅ Hybrid model saved!")

# ============================================================================
# PART 12: EVALUATION METRICS
# ============================================================================

print("\n📈 CALCULATING EVALUATION METRICS...")

def precision_at_k(recommended, relevant, k=10):
    """Precision@K"""
    recommended_k = recommended[:k]
    relevant_set = set(relevant)
    hits = len([r for r in recommended_k if r in relevant_set])
    return hits / k if k > 0 else 0

def recall_at_k(recommended, relevant, k=10):
    """Recall@K"""
    recommended_k = recommended[:k]
    relevant_set = set(relevant)
    hits = len([r for r in recommended_k if r in relevant_set])
    return hits / len(relevant_set) if len(relevant_set) > 0 else 0

def ndcg_at_k(recommended, relevant, k=10):
    """Normalized Discounted Cumulative Gain@K"""
    recommended_k = recommended[:k]
    relevant_set = set(relevant)
    
    dcg = sum([1.0 / np.log2(idx + 2) if rec in relevant_set else 0 
               for idx, rec in enumerate(recommended_k)])
    
    idcg = sum([1.0 / np.log2(idx + 2) for idx in range(min(len(relevant), k))])
    
    return dcg / idcg if idcg > 0 else 0

def evaluate_model(model, test_ratings, k=10):
    """Evaluate a recommendation model"""
    precisions = []
    recalls = []
    ndcgs = []
    
    # Sample users for faster evaluation
    test_users = test_ratings['userId'].unique()[:100]  # Sample 100 users
    
    for user_id in test_users:
        # Get user's test items (relevant items)
        relevant_items = test_ratings[test_ratings['userId'] == user_id]['movieId'].tolist()
        
        if len(relevant_items) == 0:
            continue
        
        try:
            # Get recommendations
            recs = model.recommend(user_id, n=k)
            recommended_items = [r['movieId'] for r in recs]
            
            if len(recommended_items) > 0:
                precisions.append(precision_at_k(recommended_items, relevant_items, k))
                recalls.append(recall_at_k(recommended_items, relevant_items, k))
                ndcgs.append(ndcg_at_k(recommended_items, relevant_items, k))
        except:
            continue
    
    return {
        'Precision@K': np.mean(precisions) if precisions else 0,
        'Recall@K': np.mean(recalls) if recalls else 0,
        'NDCG@K': np.mean(ndcgs) if ndcgs else 0
    }

# Evaluate models
print("\n🎯 Evaluating models on test set...")

results = {}

print("\n1. Evaluating Hybrid Model...")
results['Hybrid'] = evaluate_model(hybrid_model, test_ratings, k=10)

print("\n2. Evaluating User-User CF...")
results['User-User CF'] = evaluate_model(user_cf_model, test_ratings, k=10)

print("\n3. Evaluating Item-Item CF...")
results['Item-Item CF'] = evaluate_model(item_cf_model, test_ratings, k=10)

print("\n4. Evaluating Content-Based...")
results['Content-Based'] = evaluate_model(content_model, test_ratings, k=10)

# Display results
print("\n" + "="*70)
print("EVALUATION RESULTS (K=10)")
print("="*70)

results_df = pd.DataFrame(results).T
print(results_df.to_string())

# Save results
results_df.to_csv('evaluation_results.csv')
print("\n✅ Results saved to 'evaluation_results.csv'")

# ============================================================================
# PART 13: DIVERSITY & NOVELTY METRICS
# ============================================================================

print("\n🌈 CALCULATING DIVERSITY METRICS...")

def calculate_diversity_metrics(model, test_users, movies_df, k=10):
    """Calculate diversity and novelty metrics"""
    
    # Catalog coverage
    all_recommended = set()
    total_catalog = len(movies_df)
    
    # Intra-list diversity
    intra_diversities = []
    
    for user_id in test_users[:50]:  # Sample for speed
        try:
            recs = model.recommend(user_id, n=k)
            rec_ids = [r['movieId'] for r in recs]
            
            all_recommended.update(rec_ids)
            
            # Calculate genre diversity
            rec_genres = []
            for movie_id in rec_ids:
                movie = movies_df[movies_df['movieId'] == movie_id]
                if len(movie) > 0:
                    genres = movie.iloc[0]['genres'].split('|')
                    rec_genres.extend(genres)
            
            # Unique genres / total items
            if len(rec_ids) > 0:
                diversity = len(set(rec_genres)) / len(rec_ids)
                intra_diversities.append(diversity)
        except:
            continue
    
    catalog_coverage = len(all_recommended) / total_catalog
    avg_intra_diversity = np.mean(intra_diversities) if intra_diversities else 0
    
    return {
        'Catalog Coverage': catalog_coverage,
        'Intra-List Diversity': avg_intra_diversity
    }

# Calculate for hybrid model
test_users = test_ratings['userId'].unique()
diversity_metrics = calculate_diversity_metrics(hybrid_model, test_users, movies_df, k=10)

print("\n" + "="*70)
print("DIVERSITY METRICS (Hybrid Model)")
print("="*70)
for metric, value in diversity_metrics.items():
    print(f"{metric}: {value:.4f}")

# Save diversity metrics
with open('diversity_metrics.json', 'w') as f:
    json.dump(diversity_metrics, f, indent=2)

print("\n✅ Diversity metrics saved to 'diversity_metrics.json'")

# ============================================================================
# PART 14: EXPLAINABILITY EXAMPLES
# ============================================================================

print("\n💡 GENERATING EXPLANATION EXAMPLES...")

# Get a sample user
sample_user = train_ratings['userId'].iloc[0]

print(f"\n📌 Sample Recommendations for User {sample_user}:")
print("="*70)

# Get hybrid recommendations
hybrid_recs = hybrid_model.recommend(sample_user, n=5)

for i, rec in enumerate(hybrid_recs, 1):
    print(f"\n{i}. {rec['title']}")
    print(f"   Genres: {rec['genres']}")
    print(f"   Score: {rec['hybrid_score']:.4f}")
    print(f"   Sources: {', '.join(rec['sources'])}")
    
    # Generate explanation
    explanation = hybrid_model.explain(sample_user, rec['movieId'], rec['sources'])
    print(f"   📖 Why: {explanation}")

# ============================================================================
# PART 15: SAVE MODEL METADATA
# ============================================================================

print("\n💾 SAVING MODEL METADATA...")

metadata = {
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset_stats': {
        'total_ratings': len(ratings_df),
        'total_users': ratings_df['userId'].nunique(),
        'total_movies': ratings_df['movieId'].nunique(),
        'train_size': len(train_ratings),
        'test_size': len(test_ratings)
    },
    'models': {
        'popularity': 'PopularityRecommender',
        'user_cf': 'UserUserCF(k=20)',
        'item_cf': 'ItemItemCF(k=20)',
        'svd': 'SVD(factors=100)' if SURPRISE_AVAILABLE else 'Not trained',
        'content': 'ContentBasedRecommender',
        'hybrid': 'HybridRecommender(weights=user_cf:0.4, item_cf:0.4, content:0.2)'
    },
    'evaluation': results,
    'diversity': diversity_metrics,
    'surprise_library_available': SURPRISE_AVAILABLE
}

with open('models/model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("✅ Metadata saved to 'models/model_metadata.json'")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*70)
print("🎉 REELSENSE TRAINING PIPELINE COMPLETE!")
print("="*70)
print("\n📁 Files Generated:")
print("  ✓ models/popularity_model.pkl")
print("  ✓ models/user_cf_model.pkl")
print("  ✓ models/item_cf_model.pkl")
if SURPRISE_AVAILABLE:
    print("  ✓ models/svd_model.pkl")
else:
    print("  ⚠ models/svd_model.pkl (SKIPPED - Surprise library issue)")
print("  ✓ models/content_model.pkl")
print("  ✓ models/hybrid_model.pkl")
print("  ✓ models/model_metadata.json")
print("  ✓ evaluation_results.csv")
print("  ✓ diversity_metrics.json")
print("  ✓ eda_plots/ (6 visualization files)")

print("\n📊 Best Model Performance:")
best_model = max(results.items(), key=lambda x: x[1]['NDCG@K'])
print(f"  {best_model[0]}: NDCG@10 = {best_model[1]['NDCG@K']:.4f}")

if not SURPRISE_AVAILABLE:
    print("\n⚠️ NOTE: SVD model was skipped due to NumPy compatibility.")
    print("   To enable SVD, please restart your kernel after running this script once.")

print("\n✨ Next Step: Deploy with Streamlit UI!")
print("="*70)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.12/dist-package

⚠️ Warning: Surprise library not available: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).
   SVD model will be skipped. Other models will work fine.
✅ All libraries imported successfully!
📊 Dataset Statistics:
Ratings: 100,836 rows
Movies: 9,742 rows
Tags: 3,683 rows
Links: 9,742 rows

Users: 610
Movies rated: 9,724

📋 Sample Data:

Ratings:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Movies:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4      

In [1]:
"""
ReelSense: SVD-Only Training Script
Handles NumPy 2.x compatibility automatically
"""

import subprocess
import sys

print("="*70)
print("🎬 REELSENSE - SVD MODEL TRAINING")
print("="*70)

# ============================================================================
# STEP 1: FIX NUMPY COMPATIBILITY
# ============================================================================

print("\n🔧 Checking NumPy compatibility...")

# try:
#     import numpy as np
#     numpy_version = np.__version__
#     major_version = int(numpy_version.split('.')[0])
    
#     print(f"Current NumPy version: {numpy_version}")
    
#     if major_version >= 2:
#         print("⚠️  NumPy 2.x detected - Surprise library requires NumPy 1.x")
#         print("📥 Downgrading to NumPy 1.x...")
        
#         subprocess.check_call([
#             sys.executable, "-m", "pip", "install", 
#             "numpy<2", "--quiet"
#         ])
        
#         print("✅ NumPy downgraded successfully!")
#         print("🔄 Please RESTART your kernel/runtime and run this script again.")
#         print("\nIn Google Colab: Runtime -> Restart runtime")
#         print("In Kaggle: Session -> Restart session")
#         print("In Jupyter: Kernel -> Restart")
#         sys.exit(0)
#     else:
#         print(f"✅ NumPy {numpy_version} is compatible!")
        
# except Exception as e:
#     print(f"Error checking NumPy: {e}")
#     print("Continuing anyway...")

# ============================================================================
# STEP 2: IMPORTS
# ============================================================================

print("\n📦 Importing libraries...")

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from datetime import datetime
import pickle
import json
import os

# Try importing Surprise
try:
    from surprise import SVD, Dataset, Reader
    from surprise.model_selection import cross_validate
    from surprise import accuracy
    SURPRISE_AVAILABLE = True
    print("✅ Surprise library loaded successfully!")
except ImportError as e:
    print(f"❌ ERROR: Surprise library could not be loaded")
    print(f"   Error details: {e}")
    print("\n💡 Solution:")
    print("   1. Restart your kernel/runtime")
    print("   2. Run this script again")
    sys.exit(1)

# ============================================================================
# STEP 3: LOAD DATA
# ============================================================================

print("\n📂 Loading datasets...")

# Update these paths based on your environment
# For Kaggle:
ratings_df = pd.read_csv('/kaggle/input/movie-recommendation/ratings.csv')
movies_df = pd.read_csv('/kaggle/input/movie-recommendation/movies.csv')

# For local files, use:
# ratings_df = pd.read_csv('ratings.csv')
# movies_df = pd.read_csv('movies.csv')

print(f"✅ Loaded {len(ratings_df):,} ratings")
print(f"✅ Loaded {len(movies_df):,} movies")

# ============================================================================
# STEP 4: TRAIN-TEST SPLIT
# ============================================================================

print("\n🔀 Creating train-test split...")

# Sort by timestamp
ratings_df['timestamp'] = pd.to_datetime(ratings_df['timestamp'], unit='s')
ratings_df = ratings_df.sort_values('timestamp')

def leave_last_n_split(df, n=1):
    """Leave-last-N split per user"""
    train_list = []
    test_list = []
    
    for user_id in df['userId'].unique():
        user_data = df[df['userId'] == user_id].sort_values('timestamp')
        
        if len(user_data) > n:
            train_list.append(user_data.iloc[:-n])
            test_list.append(user_data.iloc[-n:])
        else:
            train_list.append(user_data)
    
    train_df = pd.concat(train_list, ignore_index=True)
    test_df = pd.concat(test_list, ignore_index=True)
    
    return train_df, test_df

train_ratings, test_ratings = leave_last_n_split(ratings_df, n=1)

print(f"✅ Train set: {len(train_ratings):,} ratings")
print(f"✅ Test set: {len(test_ratings):,} ratings")

# ============================================================================
# STEP 5: PREPARE DATA FOR SURPRISE
# ============================================================================

print("\n🎯 Preparing data for SVD...")

# Define rating scale
reader = Reader(rating_scale=(0.5, 5.0))

# Create Surprise dataset from train data
train_data = Dataset.load_from_df(
    train_ratings[['userId', 'movieId', 'rating']], 
    reader
)

# Build full trainset
trainset = train_data.build_full_trainset()

print(f"✅ Trainset ready: {trainset.n_users} users, {trainset.n_items} items")

# ============================================================================
# STEP 6: TRAIN SVD MODEL
# ============================================================================

print("\n🧮 Training SVD Model...")
print("   Hyperparameters:")
print("   - Factors: 100")
print("   - Epochs: 20")
print("   - Learning Rate: 0.005")
print("   - Regularization: 0.02")
print()

# Initialize SVD
svd_model = SVD(
    n_factors=100,
    n_epochs=20,
    lr_all=0.005,
    reg_all=0.02,
    verbose=True
)

# Train the model
svd_model.fit(trainset)

print("\n✅ SVD model training complete!")

# ============================================================================
# STEP 7: EVALUATION ON TRAIN SET
# ============================================================================

print("\n📊 Evaluating on train set...")

train_predictions = svd_model.test(trainset.build_testset())
train_rmse = accuracy.rmse(train_predictions, verbose=False)
train_mae = accuracy.mae(train_predictions, verbose=False)

print(f"   Train RMSE: {train_rmse:.4f}")
print(f"   Train MAE: {train_mae:.4f}")

# ============================================================================
# STEP 8: EVALUATION ON TEST SET
# ============================================================================

print("\n📊 Evaluating on test set...")

# Convert test data to Surprise format
test_data = Dataset.load_from_df(
    test_ratings[['userId', 'movieId', 'rating']], 
    reader
)

# Build testset
testset = test_data.build_full_trainset().build_testset()

# Make predictions
test_predictions = svd_model.test(testset)

# Calculate metrics
test_rmse = accuracy.rmse(test_predictions, verbose=False)
test_mae = accuracy.mae(test_predictions, verbose=False)

print(f"   Test RMSE: {test_rmse:.4f}")
print(f"   Test MAE: {test_mae:.4f}")

# ============================================================================
# STEP 9: RANKING METRICS (Precision, Recall, NDCG)
# ============================================================================

print("\n📈 Calculating ranking metrics...")

def precision_at_k(recommended, relevant, k=10):
    """Precision@K"""
    recommended_k = recommended[:k]
    relevant_set = set(relevant)
    hits = len([r for r in recommended_k if r in relevant_set])
    return hits / k if k > 0 else 0

def recall_at_k(recommended, relevant, k=10):
    """Recall@K"""
    recommended_k = recommended[:k]
    relevant_set = set(relevant)
    hits = len([r for r in recommended_k if r in relevant_set])
    return hits / len(relevant_set) if len(relevant_set) > 0 else 0

def ndcg_at_k(recommended, relevant, k=10):
    """Normalized Discounted Cumulative Gain@K"""
    recommended_k = recommended[:k]
    relevant_set = set(relevant)
    
    dcg = sum([1.0 / np.log2(idx + 2) if rec in relevant_set else 0 
               for idx, rec in enumerate(recommended_k)])
    
    idcg = sum([1.0 / np.log2(idx + 2) for idx in range(min(len(relevant), k))])
    
    return dcg / idcg if idcg > 0 else 0

# Evaluate on sample of test users
test_users = test_ratings['userId'].unique()[:100]  # Sample 100 users

precisions = []
recalls = []
ndcgs = []

print("   Evaluating 100 sample users...")

for user_id in test_users:
    # Get relevant items (items user rated in test set)
    relevant_items = test_ratings[test_ratings['userId'] == user_id]['movieId'].tolist()
    
    if len(relevant_items) == 0:
        continue
    
    # Get all items user hasn't rated in train set
    user_train_items = set(train_ratings[train_ratings['userId'] == user_id]['movieId'])
    all_items = set(train_ratings['movieId'].unique())
    unrated_items = list(all_items - user_train_items)
    
    if len(unrated_items) == 0:
        continue
    
    # Predict ratings for unrated items
    predictions_list = []
    for item_id in unrated_items:
        pred = svd_model.predict(user_id, item_id)
        predictions_list.append((item_id, pred.est))
    
    # Sort by predicted rating
    predictions_list.sort(key=lambda x: x[1], reverse=True)
    recommended_items = [item_id for item_id, _ in predictions_list[:10]]
    
    # Calculate metrics
    precisions.append(precision_at_k(recommended_items, relevant_items, k=10))
    recalls.append(recall_at_k(recommended_items, relevant_items, k=10))
    ndcgs.append(ndcg_at_k(recommended_items, relevant_items, k=10))

# Calculate average metrics
avg_precision = np.mean(precisions) if precisions else 0
avg_recall = np.mean(recalls) if recalls else 0
avg_ndcg = np.mean(ndcgs) if ndcgs else 0

print(f"\n   Precision@10: {avg_precision:.4f}")
print(f"   Recall@10: {avg_recall:.4f}")
print(f"   NDCG@10: {avg_ndcg:.4f}")

# ============================================================================
# STEP 10: CREATE WRAPPER CLASS FOR RECOMMENDATIONS
# ============================================================================

print("\n🎁 Creating recommendation wrapper...")

class SVDRecommender:
    """Wrapper class for SVD model with recommendation interface"""
    
    def __init__(self, svd_model, trainset, movies_df):
        self.svd_model = svd_model
        self.trainset = trainset
        self.movies_df = movies_df
        
        # Create mapping of internal to external IDs
        self.user_map = {iid: uid for uid, iid in trainset._raw2inner_id_users.items()}
        self.item_map = {iid: uid for uid, iid in trainset._raw2inner_id_items.items()}
    
    def recommend(self, user_id, n=10):
        """Get top N recommendations for user"""
        # Get items user has already rated
        try:
            user_inner_id = self.trainset.to_inner_uid(user_id)
            user_items = set([iid for (iid, _) in self.trainset.ur[user_inner_id]])
            user_rated_movies = [self.trainset.to_raw_iid(iid) for iid in user_items]
        except:
            user_rated_movies = []
        
        # Get all items
        all_items = self.movies_df['movieId'].unique()
        unrated_items = [item for item in all_items if item not in user_rated_movies]
        
        # Predict ratings for unrated items
        predictions = []
        for item_id in unrated_items:
            pred = self.svd_model.predict(user_id, item_id)
            predictions.append((item_id, pred.est))
        
        # Sort and get top N
        predictions.sort(key=lambda x: x[1], reverse=True)
        top_items = predictions[:n]
        
        # Format recommendations
        recommendations = []
        for movie_id, pred_rating in top_items:
            movie_info = self.movies_df[self.movies_df['movieId'] == movie_id]
            if len(movie_info) > 0:
                movie_info = movie_info.iloc[0]
                recommendations.append({
                    'movieId': movie_id,
                    'title': movie_info['title'],
                    'genres': movie_info['genres'],
                    'predicted_rating': pred_rating
                })
        
        return recommendations
    
    def explain(self, user_id, movie_id):
        """Generate explanation for recommendation"""
        return "Recommended based on latent factor analysis of your rating patterns"

# Create recommender
svd_recommender = SVDRecommender(svd_model, trainset, movies_df)

print("✅ Recommender wrapper created!")

# ============================================================================
# STEP 11: TEST RECOMMENDATIONS
# ============================================================================

print("\n🎬 Testing recommendations...")

sample_user = train_ratings['userId'].iloc[0]
recommendations = svd_recommender.recommend(sample_user, n=5)

print(f"\nTop 5 Recommendations for User {sample_user}:")
print("="*70)
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec['title']}")
    print(f"   Genres: {rec['genres']}")
    print(f"   Predicted Rating: {rec['predicted_rating']:.2f}/5.0")
    print()

# ============================================================================
# STEP 12: SAVE MODEL AND RESULTS
# ============================================================================

print("💾 Saving model and results...")

# Create output directory
os.makedirs('models', exist_ok=True)

# Save SVD model
with open('models/svd_model.pkl', 'wb') as f:
    pickle.dump(svd_model, f)
print("✅ Saved: models/svd_model.pkl")

# Save recommender wrapper
with open('models/svd_recommender.pkl', 'wb') as f:
    pickle.dump(svd_recommender, f)
print("✅ Saved: models/svd_recommender.pkl")

# Save evaluation results
results = {
    'model': 'SVD',
    'hyperparameters': {
        'n_factors': 100,
        'n_epochs': 20,
        'lr_all': 0.005,
        'reg_all': 0.02
    },
    'train_metrics': {
        'RMSE': float(train_rmse),
        'MAE': float(train_mae)
    },
    'test_metrics': {
        'RMSE': float(test_rmse),
        'MAE': float(test_mae),
        'Precision@10': float(avg_precision),
        'Recall@10': float(avg_recall),
        'NDCG@10': float(avg_ndcg)
    },
    'dataset_stats': {
        'total_ratings': len(ratings_df),
        'train_size': len(train_ratings),
        'test_size': len(test_ratings),
        'n_users': trainset.n_users,
        'n_items': trainset.n_items
    },
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

with open('svd_evaluation_results.json', 'w') as f:
    json.dump(results, indent=2, fp=f)
print("✅ Saved: svd_evaluation_results.json")

# Save as CSV for easy viewing
results_df = pd.DataFrame({
    'Metric': ['RMSE', 'MAE', 'Precision@10', 'Recall@10', 'NDCG@10'],
    'Train': [train_rmse, train_mae, '-', '-', '-'],
    'Test': [test_rmse, test_mae, avg_precision, avg_recall, avg_ndcg]
})
results_df.to_csv('svd_metrics.csv', index=False)
print("✅ Saved: svd_metrics.csv")

# ============================================================================
# FINAL SUMMARY
# ============================================================================

print("\n" + "="*70)
print("🎉 SVD MODEL TRAINING COMPLETE!")
print("="*70)

print("\n📊 Final Metrics:")
print(f"   Test RMSE: {test_rmse:.4f}")
print(f"   Test MAE: {test_mae:.4f}")
print(f"   Precision@10: {avg_precision:.4f}")
print(f"   Recall@10: {avg_recall:.4f}")
print(f"   NDCG@10: {avg_ndcg:.4f}")

print("\n📁 Generated Files:")
print("   ✅ models/svd_model.pkl")
print("   ✅ models/svd_recommender.pkl")
print("   ✅ svd_evaluation_results.json")
print("   ✅ svd_metrics.csv")

print("\n💡 Usage Example:")
print("""
# Load the model
import pickle
with open('models/svd_recommender.pkl', 'rb') as f:
    svd_rec = pickle.load(f)

# Get recommendations
recommendations = svd_rec.recommend(user_id=1, n=10)
""")

print("\n✨ All done!")
print("="*70)

🎬 REELSENSE - SVD MODEL TRAINING

🔧 Checking NumPy compatibility...

📦 Importing libraries...



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.12/dist-package

❌ ERROR: Surprise library could not be loaded
   Error details: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

💡 Solution:
   1. Restart your kernel/runtime
   2. Run this script again
Traceback (most recent call last):
  File "/tmp/ipykernel_55/4168225772.py", line 67, in <cell line: 0>
    from surprise import SVD, Dataset, Reader
  File "/usr/local/lib/python3.12/dist-packages/surprise/__init__.py", line 6, in <module>
    from .prediction_algorithms import (
  File "/usr/local/lib/python3.12/dist-packages/surprise/prediction_algorithms/__init__.py", line 23, in <module>
    from .algo_base import AlgoBase
  File "/usr/local/lib/python3.12/dist-packages/surprise/prediction_algorithms/algo_base.py", line 8, in <module>
    from .. import similarities as sims
  File "surprise/similarities.pyx", line 1, in init surprise.simila

TypeError: object of type 'NoneType' has no len()

In [2]:
!pip uninstall -y numpy
!pip install "numpy<2.0"
!pip install scikit-surprise


Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 74.4 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.26.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.47.0 which is incompatible.
google-colab 1.0.0 requires jupyter-server==2.14.0, but you have jupyter-server 2.12.5 which is incompatible.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you